# Importando as biliotecas

In [1]:
import pyqrcode
import cv2
from pyzbar.pyzbar import decode
import webbrowser as wb
import os
from PIL import Image
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs


# Definindo as funções

In [23]:
#Função top do cabeludo scalpelando o site da receita federal
def extrai_texto_nfe(url):
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')

    tr_list = soup.find_all('tr')

    text_list = []

    for tr_list_item in tr_list:
        text_list.append(tr_list_item.text)
  
    result_list = []

    for text_list_item in text_list:
        new_list = text_list_item.split('\n')
    

        new_text_list = []
     
        new_text_list.append(new_list[2])
        new_text_list.append(new_list[6])
        new_text_list.append(new_list[8])
        new_text_list.append(new_list[10])
        new_text_list.append(new_list[12])
        print(new_text_list)
        result_list.append(new_text_list)
    return result_list

#Funções para tratar os dados raspados

def produt(lista):
    
    global produto
    produto = []
    for prod in range(len(lista)):
        prdt = lista[prod][0]
        
        
        produto.append(prdt)

    
        
###

def trat_quantid(lista):
    
    global quantidade
    quantidade = []
    for prod in range(len(lista)):
        qtde = lista[prod][1].replace("Qtde.:","")
        qtde2 = qtde.replace(",",".")
        qtde3 = float(qtde2)
        
        quantidade.append(qtde3)

      
        
###

def trat_unidade(lista):
    global unidade
    unidade = []
    for prod in range(len(lista)):
        unid = lista[prod][2].replace("UN: ","")
        
        unidade.append(unid)


      
        
###

def trat_valor_unid(lista):
    global valor_unidade
    valor_unidade = []
    for prod in range(len(lista)):
        vlun = lista[prod][3].replace("Vl. Unit.:","")
        vlun2 = vlun.replace(",",".")
        vlun3 = float(vlun2)
        
        valor_unidade.append(vlun3)

       
        
        
###

def trat_valor_total(lista):
    global valor_total
    valor_total = []
    for prod in range(len(lista)):
        vltt = lista[prod][4].replace("Vl. Total","")
        vltt2 = vltt.replace(",",".")
        vltt3 = float(vltt2)
        
        valor_total.append(vltt3)

   
    
    
#Função pra preencher o DataFrame  !!! ATENÇÃO - O DF tem que ser criado anteriormente (no caso "compras2")
#- RECOMENDAÇÃO: Criaro DF dentro da função ???

def montar_df(lista):
    compras2['VALOR_TOTAL'] = valor_total
    compras2['VALOR_UNITARIO'] = valor_unidade
    compras2['UNIDADE'] = unidade
    compras2['QUANTIDADE'] = quantidade
    compras2['PRODUTO'] = produto


    
### FUNÇÃO SUPREMA ANINHANDO TODAS AS OUTRAS:


def extrair_df(lista):
    trat_quantid(lista)
    trat_unidade(lista)
    trat_valor_total(lista)
    trat_valor_unid(lista)
    produt(lista)
    montar_df(lista)

# Criando um data frame vazio (determinando as colunas)

In [24]:
compras2 = pd.DataFrame(columns = ['PRODUTO', 'QUANTIDADE', 'UNIDADE', 'VALOR_UNITARIO', 'VALOR_TOTAL'])
compras2

,PRODUTO,QUANTIDADE,UNIDADE,VALOR_UNITARIO,VALOR_TOTAL


# Lendo a foto do QR Code

In [25]:
nota = cv2.imread(r'C:\Users\kimge\OneDrive\Alecrim\notas\notat.jpeg')

In [26]:
type(nota)

numpy.ndarray

In [27]:
#Printando o objeto 'nota' decodificado
print(decode(nota))

[Decoded(data=b'http://nfe.sefaz.ba.gov.br/servicos/nfce/qrcode.aspx?p=29221273849952003173650040000483371151828395|2|1|1|69F2E189FE898E3B7CC98559E0A8D44B65A6AAEC', type='QRCODE', rect=Rect(left=218, top=338, width=586, height=584), polygon=[Point(x=218, y=343), Point(x=225, y=922), Point(x=804, y=920), Point(x=795, y=338)], quality=1, orientation='UP')]


In [28]:
### Filtrando so o link da nota
decode(nota)[0].data.decode('utf-8') ###PEGA A VISÃO !!!! IMPORTANTE

'http://nfe.sefaz.ba.gov.br/servicos/nfce/qrcode.aspx?p=29221273849952003173650040000483371151828395|2|1|1|69F2E189FE898E3B7CC98559E0A8D44B65A6AAEC'

In [29]:
#Atribuindo o link a um objeto
link_rf = decode(nota)[0].data.decode('utf-8')

# Raspando os dados da RF

In [30]:
lista_compras = extrai_texto_nfe(link_rf)

['SACOLA RETORN EKOBOM 1UN PROM', 'Qtde.:1', 'UN: UN', 'Vl. Unit.:\xa04,79', 'Vl. Total4,79']
['ENERG RED BULL 250ML PROM', 'Qtde.:1', 'UN: UN', 'Vl. Unit.:\xa06,99', 'Vl. Total6,99']
['CERV LN HEINEKEN 330ML PROM', 'Qtde.:1', 'UN: CX', 'Vl. Unit.:\xa035,88', 'Vl. Total35,88']
['CERV LN HEINEKEN 330ML PROM', 'Qtde.:1', 'UN: UN', 'Vl. Unit.:\xa05,98', 'Vl. Total5,98']
['CERV LN BUDWEISER 330ML PROM', 'Qtde.:1', 'UN: CX', 'Vl. Unit.:\xa029,94', 'Vl. Total29,94']
['CERV LN BUDWEISER 330ML PROM', 'Qtde.:2', 'UN: UN', 'Vl. Unit.:\xa04,99', 'Vl. Total9,98']
['WHISKY WHITE HORSE 1LT PROM', 'Qtde.:1', 'UN: UN', 'Vl. Unit.:\xa076,9', 'Vl. Total76,90']


# Aplicando a função Suprema

In [31]:
extrair_df(lista_compras)

In [32]:
compras2

,PRODUTO,QUANTIDADE,UNIDADE,VALOR_UNITARIO,VALOR_TOTAL
0,SACOLA RETORN EKOBOM 1UN PROM,1.0,UN,4.79,4.79
1,ENERG RED BULL 250ML PROM,1.0,UN,6.99,6.99
2,CERV LN HEINEKEN 330ML PROM,1.0,CX,35.88,35.88
3,CERV LN HEINEKEN 330ML PROM,1.0,UN,5.98,5.98
4,CERV LN BUDWEISER 330ML PROM,1.0,CX,29.94,29.94
5,CERV LN BUDWEISER 330ML PROM,2.0,UN,4.99,9.98
6,WHISKY WHITE HORSE 1LT PROM,1.0,UN,76.90,76.90


# MISSÃO CUMPRIDA !